In [1]:
%pwd

'c:\\Users\\Jorge Guillamet\\OneDrive\\lucia_tbjochina\\Data mining\\coursework-1-data\\coursework-1-data'

In [8]:
!python3 -m pip install --upgrade pip

In [14]:
!pip install pandas

^C


In [12]:
import json
import pandas

import pandas as pd
import numpy as np
from task1_2 import process_text, text_stats
from termcolor import colored
from time import perf_counter
from tqdm import tqdm

############################################################################################################
################################### Read in data ##############################################
###############################################################################
# with open('inverted_index.json', 'r') as f:
#     inverted_index = json.load(f) # tf
candidate_passages = pd.read_csv('candidate-passages-top1000.tsv', sep='\t', header=None)
test_queries = pd.read_csv('test-queries.tsv', sep='\t', header=None)
training_data = pd.read_csv('training_data.tsv', sep='\t', header=None, low_memory=False) 

# TF-IDF. Vector representation of documents and queries.
all_pid = candidate_passages[1].unique().tolist()
all_qid = test_queries[0].tolist()


ModuleNotFoundError: No module named 'pandas'

In [ ]:


print(colored('Building index..', 'green', attrs=['reverse', 'blink']))
# Create nested dictionary with terms as keys that map to another dictionary where documents are keys.
inverted_index = defaultdict(lambda: defaultdict(int))

# For each passage, record the amount of time terms appear. 
done_pid = []
for index, row in tqdm(training_data.iterrows()):
    if row[1] not in done_pid:
        vocab = process_text(row[3])
        pid = row[1]
        done_pid.append(pid)
        for term, count in vocab.items():
            inverted_index[term][pid] = count
print(colored('Success\n', 'green', attrs=['reverse', 'blink']))

# Save output
print(colored('Saving file..', 'green', attrs=['reverse', 'blink']))

In [14]:
# first row s the header

validation_data = pd.read_csv('validation_data.tsv', sep='\t', header=None, low_memory=False) #MAYEB DONT WANT LOW MEMORY FALSE
validation_data.columns = validation_data.iloc[0]
validation_data = validation_data[1:]

In [18]:
pd.set_option('display.max_colwidth', None)
validation_data[:1]

,qid,pid,queries,passage,relevancy
1,1082792,1000084,what does the golgi apparatus do to the proteins and lipids once they arrive ?,"Start studying Bonding, Carbs, Proteins, Lipids. Learn vocabulary, terms, and more with flashcards, games, and other study tools.",0.0


In [3]:
print(colored('Calculating nt and idf..', 'green', attrs=['reverse', 'blink']))
N = len(all_pid)
logN = np.log(N)
# Initialize an empty dictionary for nt and idf values
nt_idf_dict = {}
# Calculate nt and idf for each term
for term, docs in tqdm(inverted_index.items()):
    nt = len(docs)  # number of documents containing the term
    idf = logN - np.log(nt)  # optimized calculation
    nt_idf_dict[term] = [nt, idf]
print(colored('Success\n', 'green', attrs=['reverse', 'blink']))


Calculating nt and idf..


100%|██████████| 130648/130648 [00:00<00:00, 285326.30it/s]

Success



In [4]:

k1 = 1.2
k2 = 100
b = 0.75

print(colored('Calculating document lengths..', 'green', attrs=['reverse', 'blink']))
doc_lengths_dict = {}
for index, row in tqdm(candidate_passages.iterrows()):
    pid = str(row[1])
    passage = row[3]
    length_passage = text_stats(passage, remove_stopwords=True)[2]
    doc_lengths_dict[pid] = length_passage
avg_dl = np.mean(list(doc_lengths_dict.values()))
print(colored('Success\n', 'green', attrs=['reverse', 'blink']))



candidate_pids_per_qid = candidate_passages.groupby(0)[1].apply(lambda x: [str(pid) for pid in x]).to_dict()

Calculating document lengths..


189877it [01:18, 2422.76it/s]

Success



In [5]:
results_bm25 = {qid: {} for qid in all_qid} # Initialize a dictionary to store the results
print(colored('Calculating scores..', 'green', attrs=['reverse', 'blink']))
for qid in tqdm(all_qid):
    query = test_queries[test_queries[0] == qid][1].values[0]
    query_vocab = process_text(query)# tokenize query
    candidate_pids = candidate_pids_per_qid[qid]
    for pid in candidate_pids:
        bm25 = 0
        for term, tf_q in query_vocab.items():
            if term in inverted_index and pid in inverted_index[term].keys():
                doc_length = doc_lengths_dict[pid]
                tf_d = inverted_index[term][pid]
                nt = nt_idf_dict[term][0]
                bm25 = bm25 + np.log((N-nt + 0.5)/(nt+0.5))*(k1+1)*tf_d/(k1*((1-b)+b*(doc_length/avg_dl))+tf_d)*(k2+1)*tf_q/(k2+tf_q)
        results_bm25[qid][pid] = bm25
print(colored('Success\n', 'green', attrs=['reverse', 'blink']))




Calculating scores..


100%|██████████| 200/200 [00:01<00:00, 105.35it/s]

Success



In [ ]:
# Initialise empty results dataframe
final = pd.DataFrame(columns=["qid", "pid", "score"])
row_count = 0

# Loop through results and append to dataframe
print(colored('Saving bm25 scores..', 'green', attrs=['reverse', 'blink']))
for key in tqdm(results_bm25):
    
    # Sort the results by score, and select top 100
    df = pd.Series(dict(sorted(results_bm25[key].items(), key=lambda item: item[1], reverse=True))).reset_index(name="score")[0:100]
    # Build dataframe out of the results using row_count as the index
    to_concat = pd.concat([pd.DataFrame(np.repeat(key, len(df))), pd.DataFrame(df["index"].astype("str")), pd.DataFrame(df["score"])], axis=1)
    to_concat = to_concat.rename(columns={0: 'qid', 'index': 'pid'})
    # Append to final dataframe
    final = pd.concat([final, to_concat])
    # Update row count
    row_count += len(df)
print(colored('Success\n', 'green', attrs=['reverse', 'blink']))

# Expected length of the DataFrame
expected_length = 19290

# Check the length of the DataFrame
assert len(final) == expected_length, f"Error: The length of the DataFrame is {len(final)}, but it should be {expected_length}"

# If the assertion passes, save the DataFrame
final.to_csv("bm25.csv", header=None, index=False)